## Preprocessing

In [ ]:
# Run this code if the machine doesn't have tensorflow ready
#!pip install tensorflow

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://raw.githubusercontent.com/Alphaomegainfinity/deep-learning-Alphabet-Soup/main/Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_df = application_df.drop(columns =['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.

application_df[application_df.dtypes.index.tolist()].nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning

application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

count_freq = dict(application_df['APPLICATION_TYPE'].value_counts())
application_types_to_replace = dict(filter(lambda i:i[1]<528, count_freq.items()))

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning

application_df["CLASSIFICATION"].value_counts()


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1

application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts()>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = application_df[application_df['CLASSIFICATION'].isin(application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts()<1883].index)]['CLASSIFICATION']
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [81]:
# Convert categorical data to numeric with `pd.get_dummies`

application_convert = pd.get_dummies(application_df)
application_convert

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [82]:
# Split our preprocessed data into our features and target arrays

y = application_convert["IS_SUCCESSFUL"]
X_application_convert = application_convert.drop(["IS_SUCCESSFUL"], axis=1)

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X_application_convert, y, random_state = 42, stratify =y) 

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Checking the number of feature:

number_input_feature = X_train.shape[1]
number_input_feature

43

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense (units = 80, input_dim = 43,  activation ="relu")) 

# Second hidden layer

nn.add (tf.keras.layers.Dense (units = 30, activation = "relu")) 

# Output layer

nn.add (tf.keras.layers.Dense (units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model

nn.compile (loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [19]:
# Import Model Checkpoint to save the weights

from keras.callbacks import ModelCheckpoint
import os

checkpoint_path = "Resources/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname (checkpoint_path)
model_callback = tf.keras.callbacks.ModelCheckpoint (checkpoint_path, verbose =1, save_weights_only = True,save_freq = 5)

In [20]:
nn.save_weights(checkpoint_path.format (epoch = 0))

In [21]:
from keras import callbacks
# Train the model

fit_model = nn.fit (X_train_scaled, y_train, epochs=100, callbacks = [model_callback])

Streaming output truncated to the last 5000 lines.
Epoch 85: saving model to Resources/cp-0085.ckpt

Epoch 85: saving model to Resources/cp-0085.ckpt
449/804 [===============>..............] - ETA: 2s - loss: 0.5354 - accuracy: 0.7416
Epoch 85: saving model to Resources/cp-0085.ckpt
458/804 [================>.............] - ETA: 2s - loss: 0.5354 - accuracy: 0.7415
Epoch 85: saving model to Resources/cp-0085.ckpt

Epoch 85: saving model to Resources/cp-0085.ckpt
464/804 [================>.............] - ETA: 2s - loss: 0.5348 - accuracy: 0.7418
Epoch 85: saving model to Resources/cp-0085.ckpt
471/804 [================>.............] - ETA: 2s - loss: 0.5347 - accuracy: 0.7418
Epoch 85: saving model to Resources/cp-0085.ckpt

Epoch 85: saving model to Resources/cp-0085.ckpt
479/804 [================>.............] - ETA: 2s - loss: 0.5341 - accuracy: 0.7420
Epoch 85: saving model to Resources/cp-0085.ckpt
486/804 [=================>............] - ETA: 2s - loss: 0.5353 - accuracy: 0.

In [22]:
# Evaluate the model using the test data

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5605 - accuracy: 0.7269 - 418ms/epoch - 2ms/step
Loss: 0.5604827404022217, Accuracy: 0.7268804907798767


In [23]:
# Export our model to HDF5 file

nn.save("Resources/AlphabetSoupCharity_Optimisation.h5")

### Optimise the Model

In [24]:
# Increase more bin in CLASSIFICATION value counts >6

application_reduced = application_df. drop (['STATUS', 'USE_CASE'], axis =1)
application_reduced

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,Association,0,N,5000,1
1,T3,Independent,C2000,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Trust,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,Association,0,N,5000,0
34295,T4,CompanySponsored,C3000,Association,0,N,5000,0
34296,T3,CompanySponsored,C2000,Association,0,N,5000,0
34297,T5,Independent,C3000,Association,0,N,5000,1


In [25]:
# Convert categorical data to numeric with `pd.get_dummies`

application_convert_reduced = pd.get_dummies(application_reduced)
application_convert_reduced

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,108590,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,5000,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,6692,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,142590,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,5000,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,5000,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,5000,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [26]:
# Split our preprocessed data into our features and target arrays

y_reduce = application_convert_reduced["IS_SUCCESSFUL"]
X_application_reduced = application_convert_reduced.drop(["IS_SUCCESSFUL"], axis=1)

# Split the preprocessed data into a training and testing dataset

X_train_reduced, X_test_reduced, y_train_reduced, y_test_reduced = train_test_split(X_application_reduced, y_reduce, random_state = 42, stratify =y) 

In [41]:
# Create a StandardScaler instances
scaler_reduce = StandardScaler()

# Fit the StandardScaler
X_scaler_reduce = scaler_reduce.fit(X_train_reduced)

# Scale the data
X_train_scaled_reduce = X_scaler_reduce.transform(X_train_reduced)
X_test_scaled_reduce = X_scaler_reduce.transform(X_test_reduced)

In [42]:
# Checking the number of feature:

number_feature_reduce = X_train_reduced.shape[1]
number_feature_reduce

37

In [43]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_reduce = tf.keras.models.Sequential()

# First hidden layer

nn_reduce.add(tf.keras.layers.Dense (units = 100, input_dim = 37,  activation ="tanh")) 

# Second hidden layer

nn_reduce.add (tf.keras.layers.Dense (units = 50, activation = "tanh")) 

# Third hidden layer

nn_reduce.add (tf.keras.layers.Dense (units = 20, activation = "tanh")) 

# Output layer

nn_reduce.add (tf.keras.layers.Dense (units = 1, activation = "sigmoid"))

# Check the structure of the model
nn_reduce.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 100)               3800      
                                                                 
 dense_15 (Dense)            (None, 50)                5050      
                                                                 
 dense_16 (Dense)            (None, 20)                1020      
                                                                 
 dense_17 (Dense)            (None, 1)                 21        
                                                                 
Total params: 9,891
Trainable params: 9,891
Non-trainable params: 0
_________________________________________________________________


In [44]:
# Compile the model

nn_reduce.compile (loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [45]:
# Train the model

fit_model_reduce = nn_reduce.fit (X_train_scaled_reduce, y_train_reduced, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5685 - accuracy: 0.7233
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5565 - accuracy: 0.7303
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5547 - accuracy: 0.7300
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5522 - accuracy: 0.7322
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7312
Epoch 6/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5503 - accuracy: 0.7332
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7331
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5488 - accuracy: 0.7339
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7349
Epoch 10/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5471 - accuracy: 0.7359

In [53]:
# Evaluate the model using the test data

model_loss_reduce, model_accuracy_reduce = nn_reduce.evaluate(X_test_scaled_reduce,y_test_reduced,verbose=2)
print(f"Loss: {model_loss_reduce}, Accuracy: {model_accuracy_reduce}")

268/268 - 0s - loss: 0.5611 - accuracy: 0.7227 - 329ms/epoch - 1ms/step
Loss: 0.5610920786857605, Accuracy: 0.7226822376251221


### Changing testing method to ReLu

In [47]:
# Split the preprocessed data into a training and testing dataset

X_train_relu, X_test_relu, y_train_relu, y_test_relu = train_test_split(X_application_reduced, y_reduce, random_state = 42, stratify =y) 

In [48]:
# Create a StandardScaler instances
scaler_relu = StandardScaler()

# Fit the StandardScaler
X_scaler_relu = scaler_relu.fit(X_train_relu)

# Scale the data
X_train_scaled_relu = X_scaler_relu.transform(X_train_relu)
X_test_scaled_relu = X_scaler_relu.transform(X_test_relu)

In [49]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_relu = tf.keras.models.Sequential()

# First hidden layer

nn_relu.add(tf.keras.layers.Dense (units = 100, input_dim = 37,  activation ="relu")) 

# Second hidden layer

nn_relu.add (tf.keras.layers.Dense (units = 50, activation = "relu")) 

# Third hidden layer

nn_relu.add (tf.keras.layers.Dense (units = 20, activation = "relu")) 

# Output layer

nn_relu.add (tf.keras.layers.Dense (units = 1, activation = "sigmoid"))

# Check the structure of the model
nn_relu.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 100)               3800      
                                                                 
 dense_19 (Dense)            (None, 50)                5050      
                                                                 
 dense_20 (Dense)            (None, 20)                1020      
                                                                 
 dense_21 (Dense)            (None, 1)                 21        
                                                                 
Total params: 9,891
Trainable params: 9,891
Non-trainable params: 0
_________________________________________________________________


In [50]:
# Compile the model

nn_relu.compile (loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [51]:
# Train the model

fit_model_relu = nn_relu.fit (X_train_scaled_relu, y_train_relu, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5671 - accuracy: 0.7252
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5557 - accuracy: 0.7295
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5538 - accuracy: 0.7316
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5518 - accuracy: 0.7321
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5514 - accuracy: 0.7335
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5506 - accuracy: 0.7325
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5500 - accuracy: 0.7349
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7350
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7348
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5490 - accura

In [52]:
# Evaluate the model using the test data

model_loss_relu, model_accuracy_relu = nn_relu.evaluate(X_test_scaled_relu,y_test_relu,verbose=2)
print(f"Loss: {model_loss_relu}, Accuracy: {model_accuracy_relu}")

268/268 - 0s - loss: 0.5831 - accuracy: 0.7241 - 424ms/epoch - 2ms/step
Loss: 0.5830983519554138, Accuracy: 0.7240816354751587


After reduced the number of features (from 43 to 37), testing with different models, increasing number of layers as well as changing the number of neurons, epochs, the accuracy percentage didn't improve much. It can be concluded that the data may have too much noise. 

Next step, we will let the computer to run and automatically choose the best model to compare with the above methods

## Checking the model against the computer's auto selection model

In [ ]:
# Install keras_tuner
!pip install keras_tuner

In [70]:
# Import dependencies

import sklearn as skl
import keras_tuner as kt

In [83]:
# Use sklearn to split dataset

from sklearn.model_selection import train_test_split
X_train_sk, X_test_sk, y_train_sk, y_test_sk = train_test_split (X_application_convert, y, random_state = 42, stratify =y)

In [84]:
# Create scaler instance

X_scaler_sk = skl.preprocessing.StandardScaler()

# Fit the scaler

X_scaler_sk.fit (X_train_sk)

# Scale the data

X_train_scaled_sk = X_scaler_sk. transform(X_train_sk)
X_test_scaled_sk = X_scaler_sk.transform (X_test_sk)

In [85]:
# Create a method that creates a new Sequential model with hyperparameter options

def create_model(hp):
    nn_model = tf.keras.models.Sequential()

# Allow kerastuner to decide which activation function to use in hidden layers

    activation = hp. Choice('activation',['relu','tanh', 'sigmoid'])

# Allow kerastuner to decide number of neurons in first layer

    nn_model.add(tf.keras.layers.Dense(units = hp.Int ('first_units', min_value = 1, max_value = 100, step=2), activation = activation, input_dim = 43))

# Allow kerastuner to decide number of hidden layers and neurons in hidden layers

    for i in range (hp.Int ('num_layers', 1,6)):
        nn_model.add (tf.keras.layers.Dense(units = hp. Int('units_' + str(i), min_value = 1, max_value = 100, step=2), activation = activation))

    nn_model.add (tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Compile the model

    nn_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics= ["accuracy"])

    return nn_model

In [86]:
# Applying Hyperband

tuner = kt.Hyperband (create_model, objective= "val_accuracy", max_epochs= 100, hyperband_iterations= 2)

In [87]:
# Run the kerastuner seach for best hyperparameters
tuner.search(X_train_scaled_sk, y_train_sk, epochs =50, validation_data = (X_test_scaled_sk, y_test_sk))


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
mse               |?                 |activation
55                |?                 |first_units
1                 |?                 |num_layers
39                |?                 |units_0
2                 |?                 |tuner/epochs
0                 |?                 |tuner/initial_epoch
4                 |?                 |tuner/bracket
0                 |?                 |tuner/round



Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/tuner.py", line 158, in _try_build
    model = self._build_hypermodel(hp)
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/tuner.py", line 146, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "<ipython-input-85-83d3c876f97c>", line 12, in create_model
    nn_model.add(tf.keras.layers.Dense(units = hp.Int ('first_units', min_value = 1, max_value = 100, step=2), activation = activation, input_dim = 43))
  File "/usr/local/lib/python3.8/dist-packages/keras/dtensor/utils.py", line 95, in _wrap_function
    init_method(layer_instance, *args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/keras/layers/core/dense.py", line 120, in __init__
    self.activation = activations.get(activation)
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/traceback_utils.py", line 153, in error_handler
    raise e.with_traceback(filtered_tb) from 

Invalid model 0/5
Invalid model 1/5


Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/tuner.py", line 158, in _try_build
    model = self._build_hypermodel(hp)
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/tuner.py", line 146, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "<ipython-input-85-83d3c876f97c>", line 12, in create_model
    nn_model.add(tf.keras.layers.Dense(units = hp.Int ('first_units', min_value = 1, max_value = 100, step=2), activation = activation, input_dim = 43))
  File "/usr/local/lib/python3.8/dist-packages/keras/dtensor/utils.py", line 95, in _wrap_function
    init_method(layer_instance, *args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/keras/layers/core/dense.py", line 120, in __init__
    self.activation = activations.get(activation)
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/traceback_utils.py", line 153, in error_handler
    raise e.with_traceback(filtered_tb) from 

Invalid model 2/5
Invalid model 3/5


Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/tuner.py", line 158, in _try_build
    model = self._build_hypermodel(hp)
  File "/usr/local/lib/python3.8/dist-packages/keras_tuner/engine/tuner.py", line 146, in _build_hypermodel
    model = self.hypermodel.build(hp)
  File "<ipython-input-85-83d3c876f97c>", line 12, in create_model
    nn_model.add(tf.keras.layers.Dense(units = hp.Int ('first_units', min_value = 1, max_value = 100, step=2), activation = activation, input_dim = 43))
  File "/usr/local/lib/python3.8/dist-packages/keras/dtensor/utils.py", line 95, in _wrap_function
    init_method(layer_instance, *args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/keras/layers/core/dense.py", line 120, in __init__
    self.activation = activations.get(activation)
  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/traceback_utils.py", line 153, in error_handler
    raise e.with_traceback(filtered_tb) from 

Invalid model 4/5
Invalid model 5/5


RuntimeError: ignored

In [ ]:
# Get best model hyperparmeters

best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 27,
 'num_layers': 2,
 'units_0': 77,
 'units_1': 89,
 'units_2': 93,
 'units_3': 41,
 'units_4': 77,
 'units_5': 91,
 'tuner/epochs': 100,
 'tuner/initial_epoch': 34,
 'tuner/bracket': 3,
 'tuner/round': 3,
 'tuner/trial_id': '0206'}

In [ ]:
# Evaluate best model against full test data 

best_model = tuner.get_best_models(1)[0]
model_loss_sk, model_accuracy_sk = best_model.evaluate (X_test_scaled_sk, y_test_sk, verbose =2)
print (f"Loss: {model_loss_sk}, Accuracy: {model_accuracy_sk}")

268/268 - 0s - loss: 0.5566 - accuracy: 0.7282 - 385ms/epoch - 1ms/step
Loss: 0.55659019947052, Accuracy: 0.7281632423400879


#### Comment: Comparing the best model generated by a computer and the manual defined model, both models gave the similar answer : "relu" function is the best function to fit in the model with the accuracy of 72%, lost % between 55.6% and 56.1%. Besides, both models also showed 2 hidden layers were the best solution 